## Use the Azure AI Language Service
First make sure you have created an AML environment and jupyter kernel as in Notebook 1.

Test that you have selected the correct code by running the cell below.

In [2]:
import emoji
print(emoji.emojize('Python is :thumbs_up:'))

Python is 👍


- Create Azure AI Language Resource 
- Rename credentials_template.env to credentials.env (if not already)
- Fill in the endpoint and key in credentials.env 
[Detailed Instructions](https://learn.microsoft.com/en-us/azure/ai-services/language-service/language-detection/quickstart?tabs=windows&pivots=programming-language-python)

In [3]:
# Loads in environement varaibles from credentials.env. 
# Note that if you change the contents of credentials.env you will need to restart your python kernel as well as rerun this cell.
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
load_dotenv("credentials.env")

endpoint = os.environ.get("AZURE_LANGUAGE_ENDPOINT")
api_key = os.environ.get("AZURE_LANGUAGE_KEY")




In [4]:
print("Endpoint: ", endpoint)
print("API Key: ", api_key)

Endpoint:  https://language-sa.cognitiveservices.azure.com/
API Key:  be87a11bd52e4e97aa698f564026b2a4


In [5]:
# Create credentials and Language Client (note this was previously called text analytics)

credential = AzureKeyCredential(api_key)
client = TextAnalyticsClient(endpoint=endpoint, credential=credential)

#### Language Detection

Try replacing the text you give the detector to give different results.

In [6]:

documents = ["Ce document est rédigé en Français."]


response = client.detect_language(documents = documents, country_hint = 'us')[0]
print("Language: ", response.primary_language.name)

Language:  French


#### Sentiment Analysis

Try replacing the text you give the detector to give different results. 

In [7]:
documents = [
    "The food and service were unacceptable. The concierge was nice, however.",
    "It was a pretty place but evening was terrible, I got food poisoning and had to go to hospital",
    "Lovely little place, would go again"
]

result = client.analyze_sentiment(documents, show_opinion_mining=True)
doc_result = [doc for doc in result if not doc.is_error]


for document in doc_result:
    print("Document Sentiment: {}".format(document.sentiment))
    print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
        document.confidence_scores.positive,
        document.confidence_scores.neutral,
        document.confidence_scores.negative,
        ))

Document Sentiment: mixed
Overall scores: positive=0.30; neutral=0.17; negative=0.52 

Document Sentiment: negative
Overall scores: positive=0.00; neutral=0.00; negative=1.00 

Document Sentiment: positive
Overall scores: positive=1.00; neutral=0.00; negative=0.00 



#### Named Entity Extraction

Try replacing the text you give the detector to give different results. 


In [8]:
documents = ["""The Admiralty's first Hydrographer was Alexander Dalrymple, appointed in 1795 on the order of King George III and the existing charts were brought together and catalogued. The first chart Dalrymple published as Hydrographer to the Admiralty (of Quiberon Bay in Brittany) did not appear until 1800. He also issued Sailing Directions and Notices to Mariners (NMs)
"""]
result = client.recognize_entities(documents = documents)[0]

print("Named Entities:\n")
for entity in result.entities:
    print("Text: \t", entity.text, "\nCategory: \t", entity.category, "\nSubCategory: \t", entity.subcategory,
            "\nConfidence Score: \t", round(entity.confidence_score, 2), "\nLength: \t", entity.length, "\nOffset: \t", entity.offset, "\n")


Named Entities:

Text: 	 Admiralty 
Category: 	 Organization 
SubCategory: 	 None 
Confidence Score: 	 0.58 
Length: 	 9 
Offset: 	 4 

Text: 	 first 
Category: 	 Quantity 
SubCategory: 	 Ordinal 
Confidence Score: 	 0.98 
Length: 	 5 
Offset: 	 16 

Text: 	 Hydrographer 
Category: 	 PersonType 
SubCategory: 	 None 
Confidence Score: 	 0.96 
Length: 	 12 
Offset: 	 22 

Text: 	 Alexander Dalrymple 
Category: 	 Person 
SubCategory: 	 None 
Confidence Score: 	 1.0 
Length: 	 19 
Offset: 	 39 

Text: 	 1795 
Category: 	 DateTime 
SubCategory: 	 DateRange 
Confidence Score: 	 1.0 
Length: 	 4 
Offset: 	 73 

Text: 	 George III 
Category: 	 Person 
SubCategory: 	 None 
Confidence Score: 	 0.92 
Length: 	 10 
Offset: 	 99 

Text: 	 first 
Category: 	 Quantity 
SubCategory: 	 Ordinal 
Confidence Score: 	 0.97 
Length: 	 5 
Offset: 	 176 

Text: 	 Dalrymple 
Category: 	 Person 
SubCategory: 	 None 
Confidence Score: 	 1.0 
Length: 	 9 
Offset: 	 188 

Text: 	 Hydrographer 
Category: 	 PersonTy

#### PII Detection
Try replacing the text you give the detector to give different results. 


In [9]:
documents = [
        "The employee's SSN is 859-98-0987.",
        "The employee's phone number is 555-555-5555.",
        "The Admiralty's first Hydrographer was Alexander Dalrymple, appointed in 1795 on the order of King George III"
    ]
response = client.recognize_pii_entities(documents, language="en")
result = [doc for doc in response if not doc.is_error]
for doc in result:
    print("Redacted Text: {}".format(doc.redacted_text))
    for entity in doc.entities:
        print("Entity: {}".format(entity.text))
        print("\tCategory: {}".format(entity.category))
        print("\tConfidence Score: {}".format(entity.confidence_score))
        print("\tOffset: {}".format(entity.offset))
        print("\tLength: {}".format(entity.length))


Redacted Text: The ********'s SSN is ***********.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 859-98-0987
	Category: USSocialSecurityNumber
	Confidence Score: 0.85
	Offset: 22
	Length: 11
Redacted Text: The ********'s phone number is ************.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 555-555-5555
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 31
	Length: 12
Redacted Text: The *********'s first ************ was *******************, appointed in 1795 on the order of King **********
Entity: Admiralty
	Category: Organization
	Confidence Score: 0.89
	Offset: 4
	Length: 9
Entity: Hydrographer
	Category: PersonType
	Confidence Score: 0.88
	Offset: 22
	Length: 12
Entity: Alexander Dalrymple
	Category: Person
	Confidence Score: 1.0
	Offset: 39
	Length: 19
Entity: George III
	Category: Person
	Confidence Score: 0.91
	Offset: 99
	Length: 10
